In [14]:
import json
import requests
from bs4 import BeautifulSoup
import time
import pickle
from utils import *
# from newsapi import NewsApiClient # !pip install newsapi-python

In [15]:
API_KEY = "b942a2d2391e4ad8ae01c9168a5cdcc1" # salbro@mit.edu
API_KEY = "90493a497ace4b8ebe8ef5e82bfadce6" # cyrille's API key
# API_KEY = "f604134c17ae431591a2976d7c3bae55" # stephenpalbro@gmail.com api key

Let's create a massive NBA query consisting of "NBA" + all teams + top 25 players.

In [4]:
## get all NBA teams, add them to the query
with open("data/teams.txt", "r") as fp:
    teams = ["(" + team.replace("\n", "").strip().lower() + ")" for team in fp.readlines()]

query = "NBA OR " + " OR ".join(teams)
query += " OR "

# add all players to the query
query += " OR ".join(["("+name+")" for name in TOP_100_PLAYERS[:25]]) # top 25 players

# URL-ify the query
query = urlify(query)

In [6]:
param_dict = {"from": "2017-10-17", "sources":"bleacher-report",
              "pageSize":"100", "apiKey":API_KEY, "q":query} 

In [7]:
responses = get_articles(param_dict)
all_articles = []
for response in responses:
    all_articles += response['articles']

In [22]:
docs = []

for i, article in enumerate(all_articles):
    if i % 50 == 0:
        print(str(i)+"/"+str(len(all_articles)), end=" ")

    soup = BeautifulSoup(requests.get(article['url']).text, "lxml")
    content = soup.find("div", {"class":"organism contentStream"})
    if content is None:
        content_slides = soup.find_all("div", {"class":"organism contentStream slide"})
        paragraphs = []
        for content_slide in content_slides:
            paragraphs += content_slide.find_all("p")
    else:
        paragraphs = content.find_all("p")

    entire_text = "".join([p.get_text() for p in paragraphs])
    docs.append(entire_text)
    time.sleep(.300)

0/3314 50/3314 100/3314 150/3314 200/3314 250/3314 300/3314 350/3314 400/3314 450/3314 500/3314 550/3314 600/3314 650/3314 700/3314 750/3314 800/3314 850/3314 900/3314 950/3314 1000/3314 1050/3314 1100/3314 1150/3314 1200/3314 1250/3314 1300/3314 1350/3314 1400/3314 1450/3314 1500/3314 1550/3314 1600/3314 1650/3314 1700/3314 1750/3314 1800/3314 1850/3314 1900/3314 1950/3314 2000/3314 2050/3314 2100/3314 2150/3314 2200/3314 2250/3314 2300/3314 2350/3314 2400/3314 2450/3314 2500/3314 2550/3314 2600/3314 2650/3314 2700/3314 2750/3314 2800/3314 2850/3314 2900/3314 2950/3314 3000/3314 3050/3314 3100/3314 3150/3314 3200/3314 3250/3314 3300/3314 

In [ ]:
### save RAW DOCS
raw_docs = [d for d in docs]
docs_raw_filename = "docs_raw_saturday2"
with open(docs_raw_filename, 'wb') as fp:
    pickle.dump(raw_docs, fp)
    
# with open (docs_raw_filename, 'rb') as fp:
#     raw_docs = pickle.load(fp)

In [24]:
import pandas as pd

Clean docs

In [25]:
with open("data/coaches.txt", "r") as fp:
    coaches = [team.replace("\n", "").strip().lower() for team in fp.readlines()]
    
with open("data/teams.txt", "r") as fp:
    teams = [team.replace("\n", "").strip().lower() for team in fp.readlines()]
    
df = pd.read_csv("data/cities_teams.csv", header=None)

cities = [v.lower() for v in df[0].values]
team_names = [v.lower().strip() for v in df[1].values]

players_first_names, players_last_names = zip(*[p.split() for p in TOP_100_PLAYERS])

name_dict = {}
for p in TOP_100_PLAYERS:
    name_dict[p.split()[0]] = p
    name_dict[p.split()[1]] = p

name_dict["james"] = "lebron james"

In [26]:
docs_listed = [clean(article) for article in docs]
docs = [" ".join(article) for article in docs_listed]

In [28]:
def clean_doc(text):
    common = {"bron": "lebron_james", "kd": "kevin_durant", "steph": "stephen_curry", "russ": "russell_westbrook"}
    
    original_text = " ".join([word.lower() for word in text.split() if word.isalpha()])
    text = original_text # for modification
    
    for fn in players_first_names:
        wholename = name_dict[fn]
        if fn in text and wholename in original_text: # if first name and entire name is somewhere in text
            text = text.replace(wholename, "_".join(wholename.split())) # join wholename with underscore
            text = text.replace(" " + fn + " ", " " + "_".join(wholename.split()) + " ") #join the first name
    
    for ln in players_last_names:
        wholename = name_dict[ln]
        if ln in text and wholename in original_text: # if first name and entire name is somewhere in text
            text = text.replace(wholename, "_".join(wholename.split())) # join wholename with underscore
            text = text.replace(" " + ln + " ", " " + "_".join(wholename.split()) + " ") #join the first name

    for team in teams:
        if team in text:
            text = text.replace(" " + team + " ", " " + "_".join(team.split()) + " ")

    for city in cities:
        if city in text:
            text = text.replace(" " + city + " ", " " + "_".join(city.split()) + " ")

    for name in team_names:
        if name in text:
            text = text.replace(" " + name + " ", " " + "_".join(name.split()) + " ")
            
    for coach in coaches:
        last_name = coach.split()[1]
        if coach in text:
            text = text.replace(" " + coach + " ", " " + "_".join(coach.split()) + " ")
            text = text.replace(" " + last_name + " ", " " + "_".join(coach.split()) + " ")

    
    for acr, real in common.items():
        text = text.replace(" " + acr + " ", " " + real + " ")
    
    return text

docs = [clean_doc(doc) for doc in docs]

In [47]:
len(docs), len(all_articles)

(3314, 3314)

In [30]:
import pickle

In [41]:
docs_cleaned = [d for d in docs]
docs_cleaned_filename = "docs_cleaned_saturday2"
with open(docs_cleaned_filename, 'wb') as fp:
    pickle.dump(docs_cleaned, fp)
    
    
# with open (docs_cleaned_filename, 'rb') as fp:
#     cleaned_docs = pickle.load(fp)

In [32]:
def add_dict_to_file(filename, d):
    with open(filename, 'r+') as f:
        if len(f.read()) == 0:
            f.write(json.dumps(d))
        else:
            f.write('\n' + json.dumps(d))

In [37]:
article_dicts_filename = "article_dicts_saturday2"

with open(article_dicts_filename, 'r+') as f:
    f.write(json.dumps(all_articles[0]))

for ad in all_articles[1:]:
    add_dict_to_file(article_dicts_filename, ad)

In [38]:
# article_dicts_filename = "article_dicts_saturday"
# import codecs
# all_articles = []
# with codecs.open(article_dicts_filename,'rU','utf-8') as f:
#     for line in f:
#         all_articles.append(json.loads(line))

#### Let's get the articles now.

In [7]:
# %%time
# param_dict = {"from": "2017-10-17", "sources":"bleacher-report",
#               "pageSize":"100", "apiKey":API_KEY, "q":query} # you can also do "sortBy=popularity"
# set_of_docs = get_docs_from_params(param_dict)

In [6]:
# big_filename = "all_nba_from_oct17"
# with open(big_filename, 'wb') as fp:
#     pickle.dump(set_of_docs, fp)

In [24]:
# with open (big_filename, 'rb') as fp:
#     docs = pickle.load(fp)

# players only

In [8]:
# player_queries = ["(lebron james) OR lebron", "(kevin durant) OR KD", "(steph curry) OR (stephen curry)", 
#                  "(james harden) OR (harden)", "(russell westbrook) OR (westbrook)", "(anthony davis)"]
# player_docs_list = {}
# player_names = ["bron", "kd", "steph", "harden", "russel", "davis"]


# start_date = "2017-10-17" # start of 2017-2018 NBA season
# for player, name in zip(player_queries, player_names):
#     print(name)
#     param_dict = {"from": start_date, "sources":"bleacher-report",
#               "pageSize":"100", "apiKey":API_KEY, "q":player} # you can also do "sortBy=popularity"

#     player_docs = get_docs_from_params(param_dict)
#     player_docs_list[player] = player_docs

# # save to a file
# for player, name in zip(player_queries, player_names):
#     filename = name + "_from_oct17_take2"
#     with open(filename, 'wb') as fp:
#         pickle.dump(player_docs_list[player], fp)

bron
0/809...50/809...100/809...150/809...200/809...250/809...300/809...350/809...400/809...450/809...500/809...550/809...600/809...650/809...700/809...750/809...800/809...kd
0/294...50/294...100/294...150/294...200/294...250/294...steph
0/331...50/331...100/331...150/331...200/331...250/331...300/331...harden
0/195...50/195...100/195...150/195...russel
0/226...50/226...100/226...150/226...200/226...davis
0/178...50/178...100/178...150/178...

#### old stuff

Now let's scrape the comments ... having trouble with this because of the iframe. forget about it for the time being.

In [185]:
# %%time
# comments_by_doc = []
# for article in all_articles[:10]:
#     comments = []
#     url = article['url']
#     soup = BeautifulSoup(requests.get(url).text, "lxml")
#     content = soup.find_all("span", {"class":"_5mdd"})
#     if content is None:
#         print("no comment with class _5mdd")
#     for comment_span in content:
#         comments.append(comment_span.get_text())
#     comments_by_doc.append(comments)

CPU times: user 394 ms, sys: 36.2 ms, total: 430 ms
Wall time: 848 ms
